# Healthcare Insurance ETL and data cleaning

This dataset from Kaggle (https://www.kaggle.com/datasets/willianoliveiragibin/healthcare-insurance?resource=download) contains information on the relationship between personal attributes (age, gender, BMI, family size, smoking habits), geographic factors, and their impact on medical insurance charges. I will be using it to stud how these factors effect insurange costs. 

In [1]:
import numpy as np 
import pandas as pd


### Load data into a pandas dataframe

In [2]:
df = pd.read_csv('../data/insurance.csv')
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


### Check dataset size

In [3]:
df.shape

(1338, 7)

### Get data information

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


### Change datatypes to save memory and improve performance

Age can be changed to an int8 as age cannot be higher than 127. Change object to category. 

In [5]:
df = df.astype({'age':'Int8', 'sex':'category', 'smoker':'category','region':'category'})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   age       1338 non-null   Int8    
 1   sex       1338 non-null   category
 2   bmi       1338 non-null   float64 
 3   children  1338 non-null   int64   
 4   smoker    1338 non-null   category
 5   region    1338 non-null   category
 6   charges   1338 non-null   float64 
dtypes: Int8(1), category(3), float64(2), int64(1)
memory usage: 38.5 KB


Children could also be int8 as the number is not very high but it could also be category if there are not many possible values

In [6]:
df['children'].unique()

array([0, 1, 3, 2, 5, 4], dtype=int64)

In this dataset people either have 0, 1, 2, 3, 4 or 5 children. Changing the datatype to category means I could analyse how the number of children may change insurance costs later in my analysis

In [7]:
df['children'] = df['children'].astype('category')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   age       1338 non-null   Int8    
 1   sex       1338 non-null   category
 2   bmi       1338 non-null   float64 
 3   children  1338 non-null   category
 4   smoker    1338 non-null   category
 5   region    1338 non-null   category
 6   charges   1338 non-null   float64 
dtypes: Int8(1), category(4), float64(2)
memory usage: 29.5 KB


Can use describe to see some basic statistics on the different columns. Useful for charges and the categoricals, but not so useful for all categories (e.g. age)

In [8]:
df.describe(include='all').round(2)

,age,sex,bmi,children,smoker,region,charges
count,1338.0,1338,1338.00,1338.0,1338,1338,1338.00
unique,<NA>,2,NaN,6.0,2,4,NaN
top,<NA>,male,NaN,0.0,no,southeast,NaN
freq,<NA>,676,NaN,574.0,1064,364,NaN
mean,39.21,NaN,30.66,NaN,NaN,NaN,13270.42
std,14.05,NaN,6.10,NaN,NaN,NaN,12110.01
min,18.0,NaN,15.96,NaN,NaN,NaN,1121.87
25%,27.0,NaN,26.30,NaN,NaN,NaN,4740.29
50%,39.0,NaN,30.40,NaN,NaN,NaN,9382.03
75%,51.0,NaN,34.69,NaN,NaN,NaN,16639.91


### Check for null values

In [9]:
df.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

No null values found in any column

Check column names

In [10]:
df.columns

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges'], dtype='object')

Columns have sensible names, no need to change any

It does not make sense to check for duplicated values in this dataset

## Feature creation

I want to make new columns for north or south and east or west. This will allow me to show if there are any broad differences between BMI or charges between board geographical regions. 

In [11]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [12]:
def change_region_ns (n):
    '''
    Function changes anything in the south to south, and anything in the north to north
    '''
    if n == 'southwest' or n == 'southeast':
        n = 'south'
    else:
        n = 'north'
    return n

def change_region_ew (e):
    '''
    Function changes anything in the east to east, and anything in the west to west
    '''
    if e == 'southeast' or e == 'northeast':
        e = 'east'
    else:
        e = 'west'
    return e

df['region (N/S)'] = df['region'].apply(change_region_ns)
df['region (E/W)'] = df['region'].apply(change_region_ew)
df

,age,sex,bmi,children,smoker,region,charges,region (N/S),region (E/W)
0,19,female,27.900,0,yes,southwest,16884.92400,south,west
1,18,male,33.770,1,no,southeast,1725.55230,south,east
2,28,male,33.000,3,no,southeast,4449.46200,south,east
3,33,male,22.705,0,no,northwest,21984.47061,north,west
4,32,male,28.880,0,no,northwest,3866.85520,north,west
...,...,...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830,north,west
1334,18,female,31.920,0,no,northeast,2205.98080,north,east
1335,18,female,36.850,0,no,southeast,1629.83350,south,east
1336,21,female,25.800,0,no,southwest,2007.94500,south,west


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   age           1338 non-null   Int8    
 1   sex           1338 non-null   category
 2   bmi           1338 non-null   float64 
 3   children      1338 non-null   category
 4   smoker        1338 non-null   category
 5   region        1338 non-null   category
 6   charges       1338 non-null   float64 
 7   region (N/S)  1338 non-null   object  
 8   region (E/W)  1338 non-null   object  
dtypes: Int8(1), category(4), float64(2), object(2)
memory usage: 50.4+ KB


In [16]:
df = df.astype({'region (N/S)': 'category', 'region (E/W)': 'category'})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   age           1338 non-null   Int8    
 1   sex           1338 non-null   category
 2   bmi           1338 non-null   float64 
 3   children      1338 non-null   category
 4   smoker        1338 non-null   category
 5   region        1338 non-null   category
 6   charges       1338 non-null   float64 
 7   region (N/S)  1338 non-null   category
 8   region (E/W)  1338 non-null   category
dtypes: Int8(1), category(6), float64(2)
memory usage: 32.4 KB


### Make a new column to categorise BMI

According to the NHS website. For most adults, if your BMI is:

- below 18.5 – you're in the underweight range
- 18.5 to 24.9 – you're in the healthy weight range
- 25 to 29.9 – you're in the overweight range
- 30 to 39.9 – you're in the obese range
- 40 or above – you're in the severely obese range

I will add a new column that sorts BMI into these categories

In [22]:
def bmi_calculator (n):
    if n < 18.5:
        n = 'underweight'
    elif n >= 18.5 and n < 25:
        n = 'healthy weight'
    elif n >= 25 and n < 30:
        n = 'overweight'
    elif n >= 30 and n < 40:
        n = 'obese'
    else:
        n = 'severely obese'
    return n

df['bmi category'] = df['bmi'].apply(bmi_calculator)
df.head()
    

,age,sex,bmi,children,smoker,region,charges,region (N/S),region (E/W),bmi category
0,19,female,27.900,0,yes,southwest,16884.92400,south,west,overweight
1,18,male,33.770,1,no,southeast,1725.55230,south,east,obese
2,28,male,33.000,3,no,southeast,4449.46200,south,east,obese
3,33,male,22.705,0,no,northwest,21984.47061,north,west,healthy weight
4,32,male,28.880,0,no,northwest,3866.85520,north,west,overweight


In [23]:
df['bmi category'].value_counts()

bmi category
obese             616
overweight        386
healthy weight    225
severely obese     91
underweight        20
Name: count, dtype: int64